In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

# Путь к файлам
reordered_file_path = "reordered_file.csv"
kod_tibovogo_path = "kod_tipovogo.xlsx"

# Чтение файлов
df_reordered = pd.read_csv(reordered_file_path, encoding='utf-8-sig', sep=';')
df_kod_tibovogo = pd.read_excel(kod_tibovogo_path)

# Проверка считывания данных
print(f"Файл reordered_file.csv: {len(df_reordered)} строк")
print(f"Файл kod_tibovogo.xlsx: {len(df_kod_tibovogo)} строк")
print("\nПримеры данных из reordered_file.csv:")
print(df_reordered[["Название мероприятия", "Код типового мероприятия", "Тип мероприятия"]].head())
print("\nПримеры данных из kod_tibovogo.xlsx:")
print(df_kod_tibovogo[["Типовое мероприятие", "Код"]].head())

# Функция нормализации текста для лучшего сопоставления
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    # Приводим к нижнему регистру и удаляем лишние пробелы
    normalized = text.lower().strip()
    # Удаляем специальные символы, если необходимо
    # normalized = re.sub(r'[^\w\s]', '', normalized)
    return normalized

# Создаем словарь для быстрого поиска: {нормализованное название: код}
kod_mapping = {}
for _, row in df_kod_tibovogo.iterrows():
    normalized_name = normalize_text(row["Типовое мероприятие"])
    if normalized_name:
        kod_mapping[normalized_name] = row["Код"]

# Функция для нахождения наиболее подходящего соответствия
def find_best_match(event_name, reference_names, threshold=70):
    """
    Находит наиболее подходящее соответствие из списка reference_names
    для event_name с минимальным порогом соответствия threshold.
    """
    # Нормализуем название мероприятия
    normalized_name = normalize_text(event_name)
    
    # Ищем наиболее подходящее соответствие
    if normalized_name:
        # Удаляем суффикс "- Занятие X" для занятий
        if " - занятие " in normalized_name:
            normalized_name = normalized_name.split(" - занятие ")[0]
        elif " - занятие" in normalized_name:
            normalized_name = normalized_name.split(" - занятие")[0]
        
        # Находим лучшее соответствие
        matches = process.extract(
            normalized_name, 
            reference_names, 
            scorer=fuzz.token_sort_ratio,
            limit=3
        )
        
        # Если найдено соответствие с достаточным порогом
        for match, score in matches:
            if score >= threshold:
                return match
    
    # Если не найдено подходящее соответствие
    return None

# Обработка основной таблицы
print("\nОбновление кодов типовых мероприятий только для 'Программа мероприятий'...")

# Получаем список нормализованных названий из справочной таблицы
reference_names = list(kod_mapping.keys())

# Словарь для кэширования найденных соответствий для ускорения
match_cache = {}

# Счетчики для статистики
matched_count = 0
unmatched_count = 0
skipped_count = 0  # Количество пропущенных строк (не "Программа мероприятий")

# Обновляем коды типовых мероприятий
for i, row in df_reordered.iterrows():
    event_type = row["Тип мероприятия"]
    
    # Для "Дистанционное мероприятие" очищаем код типового мероприятия
    if event_type == "Дистанционное мероприятие":
        df_reordered.at[i, "Код типового мероприятия"] = ""
        skipped_count += 1
        continue
    
    # Для "Программа мероприятий" ищем соответствие
    if event_type == "Программа мероприятий":
        event_name = row["Название мероприятия"]
        
        # Проверяем, есть ли соответствие в кэше
        if event_name in match_cache:
            best_match = match_cache[event_name]
        else:
            best_match = find_best_match(event_name, reference_names)
            match_cache[event_name] = best_match
        
        # Если найдено соответствие, обновляем код
        if best_match:
            df_reordered.at[i, "Код типового мероприятия"] = kod_mapping[best_match]
            matched_count += 1
        else:
            # Если соответствие не найдено, очищаем поле (устанавливаем пустую строку)
            df_reordered.at[i, "Код типового мероприятия"] = ""
            unmatched_count += 1

# Выводим статистику
print(f"Обновлено кодов для программ мероприятий: {matched_count}")
print(f"Не найдено соответствий для программ мероприятий: {unmatched_count}")
print(f"Очищено кодов для дистанционных мероприятий: {skipped_count}")

# Записываем обновленный файл
output_file = "reordered_file_updated.csv"
df_reordered.to_csv(output_file, index=False, encoding='utf-8-sig', sep=';')
print(f"\nОбновленный файл сохранен как: {output_file}")

# Вывод примеров обновленных строк только для программ мероприятий
print("\nПримеры обновленных строк (только 'Программа мероприятий'):")
program_rows = df_reordered[df_reordered["Тип мероприятия"] == "Программа мероприятий"]
print(program_rows[["Название мероприятия", "Код типового мероприятия", "Тип мероприятия"]].head(10))

# Проверим, что коды для дистанционных мероприятий очищены
print("\nПримеры строк 'Дистанционное мероприятие' (коды должны быть пустыми):")
distance_rows = df_reordered[df_reordered["Тип мероприятия"] == "Дистанционное мероприятие"]
print(distance_rows[["Название мероприятия", "Код типового мероприятия", "Тип мероприятия"]].head(5))

Файл reordered_file.csv: 371 строк
Файл kod_tibovogo.xlsx: 246 строк

Примеры данных из reordered_file.csv:
                               Название мероприятия Код типового мероприятия  \
0              Oracle Database: Основы SQL Группа 1                   ОРСК-Б   
1  Oracle Database: Основы SQL Группа 1 - Занятие 1                   ОРСК-Б   
2  Oracle Database: Основы SQL Группа 1 - Занятие 2                   ОРСК-Б   
3  Oracle Database: Основы SQL Группа 1 - Занятие 3                   ОРСК-Б   
4  Oracle Database: Основы SQL Группа 1 - Занятие 4                   ОРСК-Б   

             Тип мероприятия  
0      Программа мероприятий  
1  Дистанционное мероприятие  
2  Дистанционное мероприятие  
3  Дистанционное мероприятие  
4  Дистанционное мероприятие  

Примеры данных из kod_tibovogo.xlsx:
                               Типовое мероприятие  \
0  1С: Управление торговлей (ред. 11.5). Логистика   
1                  Ansible: Infrastructure as Code   
2                        